In [2]:
import os
import pandas as pd

path = os.getcwd()
df_entities_spacy = pd.read_csv(os.path.join(path, "df_entities_spacy_processed.csv"))
df_entities_stanza = pd.read_csv(os.path.join(path, "df_entities_stanza_processed.csv"))

df_results = pd.read_csv("results_ner_comparison.csv")

In [3]:
df_results.head(20)
# each index represents the results for one specific bio and
# not applicable means that there were no examples of the entity in entities output by either package for that specific biography

,percent_partial_or_total_agreement_span,total_agreement_per,partial_agreement_per,total_ner_aggregate,CARDINAL,DATE,EVENT,FAC,GPE,LANGUAGE,...,MONEY,NORP,ORDINAL,ORG,PERCENT,PERSON,PRODUCT,QUANTITY,TIME,WORK_OF_ART
0,0.42,0.02,0.40,0.47,0.47,0.33,0.0,1.0,0.29,Not applicable,...,0.0,0.67,0.0,0.57,Not applicable,0.58,0.0,Not applicable,1.0,0.37
1,0.37,0.03,0.35,0.51,0.0,0.4,Not applicable,0.0,0.6,Not applicable,...,Not applicable,0.75,Not applicable,0.6,Not applicable,0.48,Not applicable,Not applicable,Not applicable,0.29
2,0.05,0.03,0.02,1.00,Not applicable,1.0,1.0,Not applicable,1.0,Not applicable,...,Not applicable,1.0,1.0,1.0,Not applicable,1.0,Not applicable,Not applicable,Not applicable,1.0
3,0.18,0.05,0.13,0.85,1.0,1.0,0.0,Not applicable,1.0,Not applicable,...,Not applicable,1.0,Not applicable,0.86,Not applicable,0.8,Not applicable,Not applicable,Not applicable,0.8
4,0.08,0.03,0.05,0.72,0.0,0.73,Not applicable,Not applicable,0.8,Not applicable,...,Not applicable,0.92,Not applicable,0.57,Not applicable,0.55,Not applicable,Not applicable,Not applicable,1.0
5,0.17,0.02,0.15,0.58,0.0,0.57,0.67,0.0,0.65,Not applicable,...,Not applicable,0.91,0.67,0.56,Not applicable,0.69,Not applicable,0.36,Not applicable,0.44
6,0.04,0.00,0.04,0.74,0.33,0.75,1.0,1.0,0.65,Not applicable,...,Not applicable,0.84,1.0,0.8,Not applicable,0.68,Not applicable,Not applicable,0.0,1.0
7,0.07,0.03,0.05,0.90,Not applicable,0.91,1.0,0.67,1.0,Not applicable,...,Not applicable,1.0,Not applicable,0.67,Not applicable,0.88,Not applicable,1.0,Not applicable,1.0
8,0.13,0.04,0.09,0.73,Not applicable,0.8,Not applicable,0.0,0.91,Not applicable,...,Not applicable,1.0,Not applicable,0.4,Not applicable,0.8,Not applicable,Not applicable,Not applicable,0.8
9,0.15,0.07,0.08,0.92,Not applicable,0.89,Not applicable,Not applicable,1.0,Not applicable,...,Not applicable,1.0,1.0,0.89,Not applicable,0.8,Not applicable,Not applicable,Not applicable,1.0


FIGURE OUT WHAT WE WANT TO SAY:

- SOME IDEAS: WHICH NERS ARE THE WORST, WHICH ARE THE BEST

- OVERALL RESULTS, MAYBE PICK SOME EXAMPLES FROM THE PROCESSED DFS FOR A QUALITATIVE ANALYSIS??